# Overview of embeddings-based retrieval

In [ ]:
#!pip install torch==2.1.0
#!pip install smart-open==5.2.1
#!pip install chromadb

# How does this work??

First we are going to understand what chroma does. We have a set of documents that is embedded and stored in the chromaDB. Next when we ask a query to a llm, the query goes through the same embedding and fetch the documents from the DB for the relevent documnets. Then both the query and the relevent documents are augumented and sent to LLM where the LLM synthesized the information from the retrived documnets to generate answer. 

In [1]:
from helper_utils import word_wrap

## this is to make the print the texts in a pretty way

In [2]:
## PdfReader is an open source pdf reader.
## Then read every page of the text and strip the white spaces
## then make sure there is no empty string in the text

from pypdf import PdfReader
reader = PdfReader("microsoft_annual_report_2022.pdf")
pdf_texts = [p.extract_text().strip() for p in reader.pages]

# Filter the empty strings
pdf_texts = [text for text in pdf_texts if text]

print(word_wrap(pdf_texts[0]))

1 Dear shareholders, colleagues, customers, and partners:  
We are
living through a period of historic economic, societal, and
geopolitical change. The world in 2022 looks nothing like 
the world in
2019. As I write this, inflation is at a 40 -year high, supply chains
are stretched, and the war in Ukraine is 
ongoing. At the same time, we
are entering a technological era with the potential to power awesome
advancements 
across every sector of our economy and society. As the
world’s largest software company, this places us at a historic

intersection of opportunity and responsibility to the world around us.
 
Our mission to empower every person and every organization on the
planet to achieve more has never been more 
urgent or more necessary.
For all the uncertainty in the world, one thing is clear: People and
organizations in every 
industry are increasingly looking to digital
technology to overcome today’s challenges and emerge stronger. And no

company is better positioned to help th

In [3]:
len(pdf_texts)

90

You can view the pdf in your browser [here](./microsoft_annual_report_2022.pdf) if you would like. 

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter


In [5]:
## The recursive text splitter splits based on separators and chunk it with chunk size of 1000 here and no overlap.

## '\n\n'.join(pdf_texts) is to use the whole texts together.

character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=0
)
character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))

print(word_wrap(character_split_texts[10]))
print(f"\nTotal chunks: {len(character_split_texts)}")

increased, due in large part to significant global datacenter
expansions and the growth in Xbox sales and usage. Despite 
these
increases, we remain dedicated to achieving a net -zero future. We
recognize that progress won’t always be linear, 
and the rate at which
we can implement emissions reductions is dependent on many factors that
can fluctuate over time.  
On the path to becoming water positive, we
invested in 21 water replenishment projects that are expected to
generate 
over 1.3  million cubic meters of volumetric benefits in nine
water basins around the world. Progress toward our zero waste

commitment included diverting more than 15,200 metric tons of solid
waste otherwise headed to landfills and incinerators, 
as well as
launching new Circular Centers to increase reuse and reduce e -waste at
our datacenters.  
We contracted to protect over 17,000 acres of land
(50% more than the land we use to operate), thus achieving our

Total chunks: 347


In [6]:
## Use Setence transformer to embed the text. It has a maximum window length of 256. So we are using windw length of 256
## to embed. It is very important to consider the contxt window as the embedding model will truncate beyond its context
## window.

token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

print(word_wrap(token_split_texts[10]))
print(f"\nTotal chunks: {len(token_split_texts)}")

increased, due in large part to significant global datacenter
expansions and the growth in xbox sales and usage. despite these
increases, we remain dedicated to achieving a net - zero future. we
recognize that progress won ’ t always be linear, and the rate at which
we can implement emissions reductions is dependent on many factors that
can fluctuate over time. on the path to becoming water positive, we
invested in 21 water replenishment projects that are expected to
generate over 1. 3 million cubic meters of volumetric benefits in nine
water basins around the world. progress toward our zero waste
commitment included diverting more than 15, 200 metric tons of solid
waste otherwise headed to landfills and incinerators, as well as
launching new circular centers to increase reuse and reduce e - waste
at our datacenters. we contracted to protect over 17, 000 acres of land
( 50 % more than the land we use to operate ), thus achieving our

Total chunks: 349


In [7]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
## This is to demonstrate how the setence embedding works
embedding_function = SentenceTransformerEmbeddingFunction()
print(embedding_function([token_split_texts[10]]))

[[0.04256269335746765, 0.03321182727813721, 0.030340097844600677, -0.0348665826022625, 0.0684165209531784, -0.08090908825397491, -0.015474393032491207, -0.0014509347965940833, -0.016744492575526237, 0.06770770251750946, -0.05054137855768204, -0.04919535666704178, 0.051399923861026764, 0.09192728996276855, -0.07177837193012238, 0.03951966390013695, -0.0128335477784276, -0.024947475641965866, -0.046228647232055664, -0.024357525631785393, 0.03394964709877968, 0.025502456352114677, 0.027317140251398087, -0.004126211628317833, -0.03633834421634674, 0.0036909214686602354, -0.027430448681116104, 0.004796730820089579, -0.02889624424278736, -0.01887071132659912, 0.03666630759835243, 0.02569584548473358, 0.03131285309791565, -0.06393438577651978, 0.05394403636455536, 0.08225348591804504, -0.04175686463713646, -0.00699579156935215, -0.023486029356718063, -0.030747944489121437, -0.002979236887767911, -0.07790941745042801, 0.009353121742606163, 0.003162869019433856, -0.02225707657635212, -0.0182946

In [8]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("microsoft_annual_report_2022", embedding_function=embedding_function)

ids = [str(i) for i in range(len(token_split_texts))]

chroma_collection.add(ids=ids, documents=token_split_texts)
chroma_collection.count()

349

In [9]:
query = "What was the total revenue?"

results = chroma_collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]

for document in retrieved_documents:
    print(word_wrap(document))
    print('\n')

revenue, classified by significant product and service offerings, was
as follows : ( in millions ) year ended june 30, 2022 2021 2020 server
products and cloud services $ 67, 321 $ 52, 589 $ 41, 379 office
products and cloud services 44, 862 39, 872 35, 316 windows 24, 761 22,
488 21, 510 gaming 16, 230 15, 370 11, 575 linkedin 13, 816 10, 289 8,
077 search and news advertising 11, 591 9, 267 8, 524 enterprise
services 7, 407 6, 943 6, 409 devices 6, 991 6, 791 6, 457 other 5, 291
4, 479 3, 768 total $ 198, 270 $ 168, 088 $ 143, 015 we have recast
certain previously reported amounts in the table above to conform to
the way we internally manage and monitor our business.


74 note 13 — unearned revenue unearned revenue by segment was as
follows : ( in millions ) june 30, 2022 2021 productivity and business
processes $ 24, 558 $ 22, 120 intelligent cloud 19, 371 17, 710 more
personal computing 4, 479 4, 311 total $ 48, 408 $ 44, 141 changes in
unearned revenue were as follows : ( in milli

In [10]:
import os  
import openai 
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
load_dotenv(Path("raj.env"))
openai.api_key=os.getenv("OPENAI_API_KEY")

openai_client = OpenAI()

In [11]:
def rag(query, retrieved_documents, model="gpt-3.5-turbo"):
    information = "\n\n".join(retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": "You are a helpful expert financial research assistant. Your users are asking questions about information contained in an annual report."
            "You will be shown the user's question, and the relevant information from the annual report. Answer the user's question using only this information."
        },
        {"role": "user", "content": f"Question: {query}. \n Information: {information}"}
    ]
    
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content

In [12]:
output = rag(query=query, retrieved_documents=retrieved_documents)

print(word_wrap(output))

The total revenue for the year ended June 30, 2022, was $198,270
million.


# Notes: 

It is a very simple idea on how to use RAG for getting query answered. 
1. Take the documnet ; chunk it using charecter splitter and then use sentence token later to be used for sentence embedding
2. Embed and store the document in chroma
3. Ask query; get the relevant data from DB
4. Ask LLM to answer the query with the information retrived.
